In [1]:
import os
from enum import Enum

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go
from mycolorpy import colorlist as mcp
from mylibs import functions as funcs
from mylibs import plots, tools
from mylibs.tools import Experiment, input_path
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator


In [2]:
%matplotlib qt

# Collect

In [4]:
# Collect data
D_mean = 15000
M_but = 74.12
M_ocm = 418.4
M_peta = 298.29
M_dmag = 198.12

but_rho = 0.7

PATH = "D:\WORKS\Diplom-work\Experiments\Viscosity"

def to_df(exp: Experiment):
    df = pd.DataFrame()
    k = 1
    df = exp.d.copy(deep=True).rename(columns={"x": "Temperature", "y": "D"})
    df["Viscosity"] = k * (df["Temperature"] + 273.15) / (df["D"] * 0.001)

    df["compound"] = exp.info["compound"]
    df["rho"] = exp.info["rho"]
    df["w_mass"] = exp.info["w"]
    df["w_vol"] = df["w_mass"] * df["rho"] / but_rho
    df["D0"] = exp.info["D0"]
    df["E"] = exp.info["E"]
    df = df[
        [
            "compound",
            "rho",
            "w_mass",
            "w_vol",
            "time",
            "Temperature",
            "D",
            "Viscosity",
            "D0",
            "E",
        ]
    ]
    return df


def collect(M_comp, comp_list):
    df = pd.concat(comp_list)
    df["x"] = (
        (df["w_mass"] / M_but)
        / (df["w_mass"] / M_but + (100 - df["w_mass"]) / M_comp)
        * 100
    )
    return df

# OCM
comp_list = []
for i in range(9):
    file = [f"{PATH}\OCM{i}\{f}" for f in os.listdir(f"{PATH}\OCM{i}") if ".hdf5" in f][
        0
    ]
    comp_list.append(to_df(Experiment.load_hdf5(file)))

ocm = collect(M_ocm, comp_list)

# PETA
comp_list = []
for i in range(5):
    file = [
        f"{PATH}\PETA{i}\{f}" for f in os.listdir(f"{PATH}\PETA{i}") if ".hdf5" in f
    ][0]
    comp_list.append(to_df(Experiment.load_hdf5(file)))
peta = collect(M_peta, comp_list)

# Butanol
butanol = to_df(Experiment.load_hdf5(f"{PATH}\BUTANOL\data_BUT_up.hdf5"))
butanol["x"] = 100
butanol["w_vol"] = 100
butanol["rho"] = but_rho

# DMAG
comp_list = []
for i in range(5):
    file = [
        f"{PATH}\DMAG{i}\{f}" for f in os.listdir(f"{PATH}\DMAG{i}") if ".hdf5" in f
    ][0]
    comp_list.append(to_df(Experiment.load_hdf5(file)))
dmag = collect(M_dmag, comp_list)

In [5]:
all_df = pd.concat([ocm, peta, butanol, dmag]).reset_index(drop=True)
DF = all_df.groupby(by=["compound", "x", "Temperature"]).mean().reset_index()

In [11]:
DF.to_csv('All_results.csv')

In [7]:
res = {}
for compound in DF["compound"].unique():
    df_comp = DF[DF["compound"] == compound]
    xs = df_comp["x"].unique()
    rhos = np.array([df_comp[df_comp["x"] == x]["rho"].mean() for x in xs]) * 1000
    print(f"{compound: >10}:", *xs.astype(int))
    print("*" * 10, *rhos.astype(int))
    res[compound] = list(zip(xs.astype(int), rhos.astype(int)))

   butanol: 100
********** 700
      dmag: 0 11 22 35 39
********** 1069 1069 1050 1029 1019
       ocm: 0 12 22 31 38 44 49 54 58
********** 1720 1580 1569 1550 1539 1530 1510 1489 1490
      peta: 0 19 30 41 49
********** 1200 1180 1159 1140 1110


# 1D

In [ ]:
class Mol(Enum):
    dmag = "dmag"
    ocm = "ocm"
    butanol = "butanol"
    peta = "peta"


class W(Enum):
    x = "x"
    mass = "w_mass"
    vol = "w_vol"

In [ ]:
# D-T plots
def plot_DT(compound: Mol, w: W):
    df = DF.copy(deep=True)
    compound_filter = (df["compound"] == compound.value) | (
        df["compound"] == Mol.butanol.value
    )

    data = df[compound_filter]
    grouped = data.groupby(by=[w.value])
    colors_list = mcp.gen_color(cmap="turbo", n=len(grouped))

    fig, ax = plt.subplots(num="DT")
    for _, group in grouped:
        x = group[w.value].mean()
        T = group["Temperature"]
        color = colors_list.pop()

        D = group["D"] / D_mean
        # ax.scatter(
        #     T,
        #     D,
        #     c=color,
        #     s=2,
        # )

        E = group["E"].mean()
        D0 = group["D0"].mean() / D_mean
        D_ols = D0 * np.exp(-E / (8.314 * (T + 273.15)))
        ax.plot(T, D_ols, label=f"{x: 5.2f} %", c=color)

        ax.set_title(f"Butanol-1, {compound.value.upper()}: Diffusion - T ({w.value})")
        ax.set_xlabel("Temperature [C]")
        ax.set_ylabel("D/D0")
    plt.legend()
    plt.show()


# D-x plots
def plot_Dx(compound: Mol, w: W):
    df = DF.copy(deep=True)
    compound_filter = (df["compound"] == compound.value) | (
        df["compound"] == Mol.butanol.value
    )
    data = df[compound_filter]
    temper_list = np.arange(15, 41, 5)
    colors_list = mcp.gen_color(cmap="turbo", n=len(temper_list))

    fig, ax = plt.subplots(num="DX")
    for T in temper_list:
        cond = (data["Temperature"] >= T - 1) & (data["Temperature"] <= T + 1)
        group = data[cond]
        group = group[[w.value, "D"]].groupby(by=w.value).mean().reset_index()

        x = group[w.value]
        color = colors_list.pop()

        D = group["D"] / D_mean
        ax.plot(x, D, c=color, label=f"{T: 3.0f} C")

        ax.set_title(f"Butanol-1, {compound.value.upper()}: Diffusion - {w.value} (T)")
        ax.set_xlabel(f"{w.value} [%]")
        ax.set_ylabel("D/D0")
    plt.legend()
    plt.show()


# E-x plots
def plot_Ex(w: W):
    df = DF.copy(deep=True)
    grouped = df.groupby(by=["compound"])
    fig, ax = plt.subplots(num="E")
    for _, group in grouped:
        name = group["compound"].unique()[0]
        group = group[["E", w.value]].groupby(by=[w.value]).mean().reset_index()
        x = group[w.value]
        E = group["E"] / 1000
        ax.plot(
            [x.iloc[0], 100],
            [E.iloc[0], 19.660651841],
            # label=f"{name}",
        )
        ax.scatter(x, E, label=f"{name}")

        ax.set_title(f"E - {w.value}")
        ax.set_xlabel(f"{w.value} [%]")
        ax.set_ylabel("E [kJ]")
    plt.legend()
    plt.show()


def plot_rho(w: W):
    df = DF.copy(deep=True)
    grouped = df.groupby(by=["compound"])
    fig, ax = plt.subplots()

    for _, group in grouped:
        name = group["compound"].unique()[0]
        group = group[["rho", w.value]].groupby(by=[w.value]).mean().reset_index()
        x = group[w.value]
        E = group["rho"]
        # ax.plot(
        #     [w.iloc[0],w.iloc[-1]],
        #     [E.iloc[0],E.iloc[-1]],
        #     # label=f"{name}",
        # )
        ax.scatter(x, E, label=f"{name}")

        ax.set_title(f"Rho - {w.value}")
        ax.set_xlabel(f"{w.value} [%]")
        ax.set_ylabel("rho [g/ml]")
    plt.legend()
    plt.show()

In [ ]:
plot_rho(W.vol)

# 2D

In [ ]:
# 2D surface
compound_filter = DF["compound"] == "ocm"
# |(df['compound'] == 'butanol')
ocm = DF[compound_filter]

xy = ocm[["Temperature", "w_mass"]]
z = ocm["D"]
X = np.linspace(min(xy["Temperature"]), max(xy["Temperature"]))
Y = np.linspace(min(xy["w_mass"]), max(xy["w_mass"]))
X, Y = np.meshgrid(X, Y)
f = LinearNDInterpolator(xy, z)
Z = f(X, Y)
plt.pcolormesh(X, Y, Z)

In [ ]:
fig = go.Figure()
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    scene=dict(
        xaxis=dict(title="Temperature [C]"),
        yaxis=dict(title="fi [%]"),
        zaxis=dict(title="Diffusion"),
    ),
)
quality = 21

# OCM
compound_filter = DF["compound"] == "ocm"
# |(df['compound'] == 'butanol')

ocm = DF[compound_filter]
xy = ocm[["Temperature", "w_mass"]]
x = ocm["Temperature"]
y = ocm["w_mass"]
z = ocm["D"] / D_mean


f = LinearNDInterpolator(xy, z)

x1 = np.linspace(x.min(), x.max(), quality)
y1 = np.linspace(y.min(), y.max(), quality)
X, Y = np.meshgrid(x1, y1)

Z = f(X, Y)
R = Z
fig.add_trace(
    go.Surface(
        name="OCM-2",
        x=x1,
        y=y1,
        z=Z,
        colorscale="Reds",
        surfacecolor=R,
        showscale=False,
        showlegend=True,
        opacity=0.7,
        lightposition=dict(x=50, y=100, z=1.0),
    )
)

# PETA
compound_filter = DF["compound"] == "peta"
# |(df['compound'] == 'butanol')
ocm = DF[compound_filter]

xy = ocm[["Temperature", "w_mass"]]
x = ocm["Temperature"]
y = ocm["w_mass"]
z = ocm["D"] / D_mean

fz = CloughTocher2DInterpolator(xy, z)

x1 = np.linspace(x.min(), x.max(), quality)
y1 = np.linspace(y.min(), y.max(), quality)
X, Y = np.meshgrid(x1, y1)


Z = fz(X, Y)

fig.add_trace(
    go.Surface(
        name="PETA",
        x=x1,
        y=y1,
        z=Z,
        colorscale="Blues",
        surfacecolor=-R,
        showscale=False,
        showlegend=True,
        lightposition=dict(x=50, y=100, z=1.0),
    )
)
fig.show()

# Other plots

In [ ]:
folders = [
    f"{PATH}\{i}"
    for i in os.listdir(PATH)
    if os.path.isdir(f"{PATH}\{i}") and "PETA" in i
]

for folder in folders:
    exp = Experiment()

    file = [file for file in os.listdir(folder) if ".hdf5" in file][0]

    exp.load_hdf5(input_path(f"{folder}\{file}"))
    exp.apply(funcs.C_to_K)
    exp.apply(funcs.nu_D)
    exp.apply(funcs.K_to_C)
    # exp.folder='.'

    plots.temperature_plot(
        exp,
        title="Temperature",
        xlabel="Temperature [C]",
        ylabel="Viscosity [cP]",
        # ylabel='Diffusion [m2/s]',
        interactive=True,
        save_folder=exp.folder,
    )

In [ ]:
exp = Experiment()
exp.load_hdf5(input_path())
# exp.apply(funcs.C_to_K)
# exp.apply(funcs.nu_D)
# exp.apply(funcs.K_to_C)

info, result, func = tools.regress(exp)
exp.set_info(**info)
x = np.linspace(13, 42, 100) + 273.15
ols_res = tools.Experiment(
    pd.DataFrame({"x": x, "y": func(x), "time": x * 0}),
    "interpolated",
)

ols_res.apply(funcs.K_to_C)

plots.comparation_plot(
    exp,
    ols_res,
    title="OLS_Diffusion",
    xlabel="Temperature [C]",
    ylabel="D [m2/s]",
    interactive=True,
    save_folder=exp.folder,
)
# plots.temperature_plot(
#     exp,
#     title='Viscosity',
#     xlabel='Temperature [C]',
#     ylabel='Viscosity [cP]',
#     # ylabel='Diffusion [m2/s]',
#     interactive=True,
#     save_folder=exp.folder,
# )

# Analyse